In [ ]:
!pip install torchvision torch --quiet
#LENET-5
import torch
from torch import nn
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import numpy as np
import os

# -------------------------
# Model definition
# -------------------------
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(256, 120)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        y = self.pool1(self.relu1(self.conv1(x)))
        y = self.pool2(self.relu2(self.conv2(y)))
        y = y.view(y.shape[0], -1)
        y = self.relu3(self.fc1(y))
        y = self.relu4(self.fc2(y))
        y = self.fc3(y)
        return y

# -------------------------
# Training setup
# -------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

batch_size = 256
train_dataset = MNIST(root='./data', train=True, transform=ToTensor(), download=True)
test_dataset = MNIST(root='./data', train=False, transform=ToTensor(), download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model = Model().to(device)
sgd = SGD(model.parameters(), lr=0.1)
loss_fn = CrossEntropyLoss()

all_epoch = 20
prev_acc = 0

# -------------------------
# Training loop
# -------------------------
for current_epoch in range(all_epoch):
    model.train()
    for train_x, train_label in train_loader:
        train_x, train_label = train_x.to(device), train_label.to(device)
        sgd.zero_grad()
        predict_y = model(train_x)
        loss = loss_fn(predict_y, train_label)
        loss.backward()
        sgd.step()

    # Evaluation
    model.eval()
    all_correct_num = 0
    all_sample_num = 0

    for test_x, test_label in test_loader:
        test_x, test_label = test_x.to(device), test_label.to(device)
        output = model(test_x)
        prediction = torch.argmax(output, dim=1)
        all_correct_num += (prediction == test_label).sum().item()
        all_sample_num += test_label.size(0)

    acc = all_correct_num / all_sample_num
    print(f"Epoch {current_epoch+1}, accuracy: {acc:.4f}")

    # Save model
    if not os.path.isdir("models"):
        os.mkdir("models")
    torch.save(model.state_dict(), f"models/mnist_{acc:.3f}.pth")

    # Stop if accuracy stops changing
    if abs(acc - prev_acc) < 1e-4:
        break
    prev_acc = acc

print("Training completed.")


Device: cpu


100%|██████████| 9.91M/9.91M [00:00<00:00, 18.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 484kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.49MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.14MB/s]


Epoch 1, accuracy: 0.9185
Epoch 2, accuracy: 0.9546
Epoch 3, accuracy: 0.9698
Epoch 4, accuracy: 0.9772
Epoch 5, accuracy: 0.9804
Epoch 6, accuracy: 0.9745
Epoch 7, accuracy: 0.9792
Epoch 8, accuracy: 0.9685
Epoch 9, accuracy: 0.9855
Epoch 10, accuracy: 0.9862
Epoch 11, accuracy: 0.9852
Epoch 12, accuracy: 0.9877
Epoch 13, accuracy: 0.9627
Epoch 14, accuracy: 0.9838
Epoch 15, accuracy: 0.9877
Epoch 16, accuracy: 0.9861
Epoch 17, accuracy: 0.9820
Epoch 18, accuracy: 0.9867
Epoch 19, accuracy: 0.9889
Epoch 20, accuracy: 0.9784
Training completed.


In [ ]:
# ================================================================
#  FULL ALEXNET TRAINING SCRIPT FOR MNIST (COLAB READY)
# ================================================================

import torch
import torch.nn as nn
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Resize
import numpy as np
import os

# ================================================================
#  AlexNet Model (adjusted for MNIST 1×224×224)
# ================================================================
class AlexNetMNIST(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNetMNIST, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),  # changed 3→1 for MNIST
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),

            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),

            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# ================================================================
#  Setup
# ================================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

batch_size = 256
epochs = 20

# Transform MNIST for AlexNet size
transform = Compose([
    Resize((224, 224)),  # AlexNet size
    ToTensor()
])

# Datasets
train_dataset = MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = MNIST(root="./data", train=False, transform=transform, download=True)

# Loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Model + optimizer + loss
model = AlexNetMNIST().to(device)
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_fn = CrossEntropyLoss()

# ================================================================
#  Training Loop
# ================================================================
prev_acc = 0.0

for epoch in range(epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

    # ------------------------------
    # Evaluation
    # ------------------------------
    model.eval()
    correct = 0
    total = 0

    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1}/{epochs}   Accuracy: {acc:.4f}")

    # Save model
    if not os.path.isdir("models"):
        os.mkdir("models")
    torch.save(model.state_dict(), f"models/alexnet_mnist_{acc:.4f}.pth")

    # Early stopping if accuracy stops improving
    if abs(acc - prev_acc) < 1e-4:
        print("Early stopping triggered.")
        break
    prev_acc = acc

print("Training complete!")


Using device: cpu
Epoch 1/20   Accuracy: 0.1135
Epoch 2/20   Accuracy: 0.8579
Epoch 3/20   Accuracy: 0.9702
Epoch 4/20   Accuracy: 0.9811
Epoch 5/20   Accuracy: 0.9879
Epoch 6/20   Accuracy: 0.9884
Epoch 7/20   Accuracy: 0.9908


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# ----------------------------------------------------------
# DEVICE
# ----------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using:", device)

# ----------------------------------------------------------
# TINY CNN MODEL (very fast, ~99% accuracy)
# ----------------------------------------------------------
class TinyCNN(nn.Module):
    def __init__(self):
        super(TinyCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # 1 → 16
            nn.ReLU(),
            nn.MaxPool2d(2),  # 28→14

            nn.Conv2d(16, 32, kernel_size=3, padding=1), # 16 → 32
            nn.ReLU(),
            nn.MaxPool2d(2),  # 14→7

            nn.Flatten(),
            nn.Linear(32 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)

# ----------------------------------------------------------
# DATASET & DATALOADER
# ----------------------------------------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_data  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=1000)

# ----------------------------------------------------------
# TRAIN / TEST FUNCTIONS
# ----------------------------------------------------------
def train_one_epoch(model, loader, optimizer, loss_fn):
    model.train()
    correct, total = 0, 0

    for X, y in loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    return correct / total


def test_model(model, loader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total


# ----------------------------------------------------------
# TRAINING TINYCNN
# ----------------------------------------------------------
model = TinyCNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print("\n🔥 Training TinyCNN\n")

for epoch in range(3):   # change epochs if needed
    train_acc = train_one_epoch(model, train_loader, optimizer, loss_fn)
    test_acc  = test_model(model, test_loader)
    print(f"Epoch {epoch+1}: Train Acc = {train_acc:.4f}, Test Acc = {test_acc:.4f}")


Using: cpu


100%|██████████| 9.91M/9.91M [00:00<00:00, 18.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 495kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.70MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.48MB/s]



🔥 Training TinyCNN

Epoch 1: Train Acc = 0.9462, Test Acc = 0.9790
Epoch 2: Train Acc = 0.9829, Test Acc = 0.9886
Epoch 3: Train Acc = 0.9883, Test Acc = 0.9890


In [ ]:
# =============================
#  Micro-CNN for MNIST
#  Extremely lightweight model
# =============================

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# -------------------------
# Hyperparameters
# -------------------------
batch_size = 64
epochs = 5
lr = 0.001

# -------------------------
# MNIST Dataset
# -------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_set = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_set = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# -------------------------
# Micro-CNN Model
# -------------------------
class MicroCNN(nn.Module):
    def __init__(self):
        super(MicroCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, kernel_size=3, padding=1)  # out: 4 × 28 × 28
        self.conv2 = nn.Conv2d(4, 8, kernel_size=3, padding=1)  # out: 8 × 28 × 28
        self.pool = nn.MaxPool2d(2)  # out: 8 × 14 × 14

        self.fc1 = nn.Linear(8 * 14 * 14, 32)
        self.fc2 = nn.Linear(32, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = MicroCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# -------------------------
# Optimizer and Loss
# -------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# -------------------------
# Training Loop
# -------------------------
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")

# -------------------------
# Testing
# -------------------------
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")


Epoch [1/5], Loss: 0.2661
Epoch [2/5], Loss: 0.0842
Epoch [3/5], Loss: 0.0642
Epoch [4/5], Loss: 0.0524
Epoch [5/5], Loss: 0.0440
Accuracy on test set: 98.40%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# ---------- Dataset ----------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_loader = DataLoader(datasets.MNIST("./data", train=True, download=True, transform=transform), batch_size=64, shuffle=True)
test_loader = DataLoader(datasets.MNIST("./data", train=False, download=True, transform=transform), batch_size=64, shuffle=False)


# ---------- Depthwise Separable Convolution ----------
class DSConv(nn.Module):
    def __init__(self):
        super().__init__()

        # depthwise conv: groups = channels
        self.depth1 = nn.Conv2d(1, 1, kernel_size=3, padding=1, groups=1)
        self.point1 = nn.Conv2d(1, 8, kernel_size=1)

        self.depth2 = nn.Conv2d(8, 8, kernel_size=3, padding=1, groups=8)
        self.point2 = nn.Conv2d(8, 16, kernel_size=1)

        self.pool = nn.MaxPool2d(2)
        self.relu = nn.ReLU()

        self.fc = nn.Linear(16 * 14 * 14, 10)

    def forward(self, x):
        x = self.relu(self.point1(self.depth1(x)))
        x = self.pool(self.relu(self.point2(self.depth2(x))))
        x = x.view(x.size(0), -1)
        return self.fc(x)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DSConv().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# ---------- Training ----------
for epoch in range(5):
    model.train()
    total_loss = 0
    for img, label in train_loader:
        img, label = img.to(device), label.to(device)
        optimizer.zero_grad()
        out = model(img)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss = {total_loss/len(train_loader):.4f}")

# ---------- Testing ----------
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for img, label in test_loader:
        img, label = img.to(device), label.to(device)
        pred = model(img).argmax(1)
        total += label.size(0)
        correct += (pred == label).sum().item()

print("Accuracy:", 100 * correct / total)


Epoch 1, Loss = 0.3480
Epoch 2, Loss = 0.1653
Epoch 3, Loss = 0.1265
Epoch 4, Loss = 0.0927
Epoch 5, Loss = 0.0778
Accuracy: 97.34


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# ---------- Dataset ----------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = DataLoader(datasets.MNIST("./data", train=True, download=True, transform=transform), batch_size=64, shuffle=True)
test_loader = DataLoader(datasets.MNIST("./data", train=False, download=True, transform=transform), batch_size=64, shuffle=False)


# ---------- Fire Block ----------
class Fire(nn.Module):
    def __init__(self, in_ch, squeeze_ch, expand_ch):
        super().__init__()
        self.squeeze = nn.Conv2d(in_ch, squeeze_ch, kernel_size=1)
        self.expand1x1 = nn.Conv2d(squeeze_ch, expand_ch, kernel_size=1)
        self.expand3x3 = nn.Conv2d(squeeze_ch, expand_ch, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.squeeze(x))
        return self.relu(torch.cat([self.expand1x1(x), self.expand3x3(x)], 1))


# ---------- SqueezeTiny ----------
class SqueezeTiny(nn.Module):
    def __init__(self):
        super().__init__()

        self.fire1 = Fire(1, 4, 4)   # Output: 8 channels
        self.fire2 = Fire(8, 4, 4)   # Output: 8 channels
        self.pool = nn.MaxPool2d(2)

        self.fc = nn.Linear(8 * 14 * 14, 10)

    def forward(self, x):
        x = self.fire1(x)
        x = self.pool(self.fire2(x))
        x = x.view(x.size(0), -1)
        return self.fc(x)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SqueezeTiny().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# ---------- Training ----------
for epoch in range(5):
    model.train()
    total_loss = 0
    for img, label in train_loader:
        img, label = img.to(device), label.to(device)
        optimizer.zero_grad()
        out = model(img)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss = {total_loss/len(train_loader):.4f}")

# ---------- Testing ----------
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for img, label in test_loader:
        img, label = img.to(device), label.to(device)
        pred = model(img).argmax(1)
        total += label.size(0)
        correct += (pred == label).sum().item()

print("Accuracy:", 100 * correct / total)


Epoch 1, Loss = 0.3708
Epoch 2, Loss = 0.1344
Epoch 3, Loss = 0.0948
Epoch 4, Loss = 0.0750
Epoch 5, Loss = 0.0650
Accuracy: 97.69


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# ---------- Dataset ----------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = DataLoader(datasets.MNIST("./data", train=True, download=True, transform=transform), batch_size=64, shuffle=True)
test_loader = DataLoader(datasets.MNIST("./data", train=False, download=True, transform=transform), batch_size=64, shuffle=False)


# ---------- Residual Block ----------
class ResBlock(nn.Module):
    def __init__(self, ch):
        super().__init__()
        self.conv1 = nn.Conv2d(ch, ch, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(ch)
        self.conv2 = nn.Conv2d(ch, ch, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(ch)
        self.relu = nn.ReLU()

    def forward(self, x):
        identity = x
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        return self.relu(x + identity)


# ---------- TinyResNet ----------
class TinyResNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv = nn.Conv2d(1, 8, 3, padding=1)
        self.relu = nn.ReLU()

        self.block1 = ResBlock(8)
        self.block2 = ResBlock(8)

        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(8 * 14 * 14, 10)

    def forward(self, x):
        x = self.relu(self.conv(x))
        x = self.block1(x)
        x = self.pool(self.block2(x))
        x = x.view(x.size(0), -1)
        return self.fc(x)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TinyResNet().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# ---------- Training ----------
for epoch in range(5):
    model.train()
    total_loss = 0
    for img, label in train_loader:
        img, label = img.to(device), label.to(device)
        optimizer.zero_grad()
        out = model(img)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss = {total_loss/len(train_loader):.4f}")

# ---------- Testing ----------
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for img, label in test_loader:
        img, label = img.to(device), label.to(device)
        pred = model(img).argmax(1)
        total += label.size(0)
        correct += (pred == label).sum().item()

print("Accuracy:", 100 * correct / total)


Epoch 1, Loss = 0.1516
Epoch 2, Loss = 0.0608
Epoch 3, Loss = 0.0482
Epoch 4, Loss = 0.0407
Epoch 5, Loss = 0.0344
Accuracy: 98.43


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


# ---------------------------
# MNIST Dataset
# ---------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = DataLoader(
    datasets.MNIST("./data", train=True, download=True, transform=transform),
    batch_size=64, shuffle=True
)
test_loader = DataLoader(
    datasets.MNIST("./data", train=False, download=True, transform=transform),
    batch_size=64, shuffle=False
)


# ---------------------------
# Inception Block (Tiny)
# ---------------------------
class InceptionTiny(nn.Module):
    def __init__(self, in_ch, out_1x1, out_3x3, out_5x5):
        super().__init__()

        self.branch1 = nn.Sequential(
            nn.Conv2d(in_ch, out_1x1, kernel_size=1),
            nn.ReLU()
        )

        self.branch3 = nn.Sequential(
            nn.Conv2d(in_ch, out_3x3, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.branch5 = nn.Sequential(
            nn.Conv2d(in_ch, out_5x5, kernel_size=5, padding=2),
            nn.ReLU()
        )

    def forward(self, x):
        return torch.cat([self.branch1(x),
                          self.branch3(x),
                          self.branch5(x)], dim=1)


# ---------------------------
# Tiny GoogLeNet for MNIST
# ---------------------------
class TinyGoogLeNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.stem = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU()
        )

        # One tiny inception block — outputs: 8 + 8 + 8 = 24 channels
        self.inception1 = InceptionTiny(16, 8, 8, 8)

        self.pool = nn.MaxPool2d(2)  # 28 → 14

        # Second tiny inception block
        self.inception2 = InceptionTiny(24, 12, 12, 12)  # outputs 36

        self.fc = nn.Linear(36 * 14 * 14, 10)

    def forward(self, x):
        x = self.stem(x)
        x = self.inception1(x)
        x = self.pool(x)
        x = self.inception2(x)

        x = x.view(x.size(0), -1)
        return self.fc(x)


# ---------------------------
# Training Setup
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TinyGoogLeNet().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


# ---------------------------
# Training Loop
# ---------------------------
for epoch in range(5):
    total_loss = 0
    model.train()
    for img, label in train_loader:
        img, label = img.to(device), label.to(device)

        optimizer.zero_grad()
        out = model(img)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Loss = {total_loss/len(train_loader):.4f}")


# ---------------------------
# Test Accuracy
# ---------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for img, label in test_loader:
        img, label = img.to(device), label.to(device)
        pred = model(img).argmax(1)
        correct += (pred == label).sum().item()
        total += label.size(0)

print("Accuracy:", 100 * correct / total)


Epoch 1: Loss = 0.1504
Epoch 2: Loss = 0.0438
Epoch 3: Loss = 0.0296
Epoch 4: Loss = 0.0213
Epoch 5: Loss = 0.0168
Accuracy: 99.04


In [ ]:
# Colab-ready: All 8 ultra-lightweight MNIST models in one notebook
# Run this cell start-to-finish.

!pip install -q torch torchvision --upgrade

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time
from functools import reduce
import operator

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# --------------------------
# Hyperparams (tweak if needed)
# --------------------------
BATCH_SIZE = 128
EPOCHS = 5
LR = 1e-3

# --------------------------
# MNIST dataset (28x28 grayscale)
# --------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_ds = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_ds  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=1000, shuffle=False, num_workers=2, pin_memory=True)

# --------------------------
# Utility: count params
# --------------------------
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# --------------------------
# 1) NanoCNN (~20K params)
# --------------------------
class NanoCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),  # 1->8, 28x28
            nn.ReLU(),
            nn.MaxPool2d(2),                # 8x14x14
            nn.Conv2d(8, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                # 16x7x7
            nn.Flatten()
        )
        self.classifier = nn.Sequential(
            nn.Linear(16*7*7, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

# --------------------------
# 2) PicoCNN (~8K params)
# --------------------------
class PicoCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 6, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),    # 6x14x14
            nn.Conv2d(6, 12, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),    # 12x7x7
            nn.Flatten(),
            nn.Linear(12*7*7, 40),
            nn.ReLU(),
            nn.Linear(40, 10)
        )
    def forward(self, x):
        return self.net(x)

# --------------------------
# 3) MiniMobileNet (simple DSConv blocks)
# --------------------------
def depthwise_conv(in_ch, out_ch, stride=1):
    return nn.Sequential(
        nn.Conv2d(in_ch, in_ch, kernel_size=3, stride=stride, padding=1, groups=in_ch, bias=False),
        nn.BatchNorm2d(in_ch),
        nn.ReLU(),
        nn.Conv2d(in_ch, out_ch, kernel_size=1, bias=False),
        nn.BatchNorm2d(out_ch),
        nn.ReLU()
    )

class MiniMobileNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1, bias=False), nn.BatchNorm2d(8), nn.ReLU()
        )
        self.ds1 = depthwise_conv(8, 16, stride=1)
        self.pool = nn.MaxPool2d(2)
        self.ds2 = depthwise_conv(16, 24, stride=1)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(24*7*7, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        x = self.stem(x)
        x = self.ds1(x)
        x = self.pool(x)   # 16x14x14
        x = self.ds2(x)    # 24x14x14
        x = self.pool(x)   # 24x7x7
        return self.fc(x)

# --------------------------
# 4) MobileNet-Nano (smaller channels)
# --------------------------
class MobileNetNano(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(1, 6, 3, padding=1, bias=False),
            nn.BatchNorm2d(6),
            nn.ReLU()
        )
        self.dw_pw = nn.Sequential(
            nn.Conv2d(6, 6, 3, padding=1, groups=6, bias=False),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.Conv2d(6, 12, 1, bias=False),
            nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.MaxPool2d(2), # 12x14x14

            nn.Conv2d(12, 12, 3, padding=1, groups=12, bias=False),
            nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.Conv2d(12, 20, 1, bias=False),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.MaxPool2d(2)  # 20x7x7
        )
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(20*7*7, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        x = self.stem(x)
        x = self.dw_pw(x)
        return self.head(x)

# --------------------------
# 5) TinyShuffleNet (simple channel shuffle block)
# --------------------------
def channel_shuffle(x, groups):
    batchsize, num_channels, height, width = x.size()
    channels_per_group = num_channels // groups
    # reshape
    x = x.view(batchsize, groups, channels_per_group, height, width)
    x = torch.transpose(x, 1, 2).contiguous()
    x = x.view(batchsize, -1, height, width)
    return x

class ShuffleUnit(nn.Module):
    def __init__(self, in_c, out_c, groups=2):
        super().__init__()
        mid_c = out_c // 2
        self.groups = groups
        self.conv_reduce = nn.Conv2d(in_c, mid_c, 1, bias=False)
        self.dw = nn.Conv2d(mid_c, mid_c, 3, padding=1, groups=mid_c, bias=False)
        self.conv_expand = nn.Conv2d(mid_c, out_c, 1, bias=False)
        self.bn = nn.BatchNorm2d(out_c)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv_reduce(x)
        x = self.dw(x)
        x = self.conv_expand(x)
        x = self.bn(x)
        x = self.relu(x)
        x = channel_shuffle(x, groups=self.groups)
        return x

class TinyShuffleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Conv2d(1, 12, 3, padding=1)
        self.stage1 = ShuffleUnit(12, 24)
        self.pool  = nn.MaxPool2d(2)  # 24x14x14
        self.stage2 = ShuffleUnit(24, 32)
        self.pool2  = nn.MaxPool2d(2) # 32x7x7
        self.fc = nn.Sequential(nn.Flatten(), nn.Linear(32*7*7, 64), nn.ReLU(), nn.Linear(64, 10))
    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.pool(x)
        x = self.stage2(x)
        x = self.pool2(x)
        return self.fc(x)

# --------------------------
# 6) MicroDenseNet (very small DenseNet-like)
# --------------------------
class TinyDenseLayer(nn.Module):
    def __init__(self, in_ch, growth):
        super().__init__()
        self.bn = nn.BatchNorm2d(in_ch)
        self.conv = nn.Conv2d(in_ch, growth, 3, padding=1, bias=False)
        self.relu = nn.ReLU()
    def forward(self, x):
        out = self.conv(self.relu(self.bn(x)))
        return torch.cat([x, out], 1)

class MicroDenseNet(nn.Module):
    def __init__(self, growth=8):
        super().__init__()
        self.init = nn.Conv2d(1, 12, 3, padding=1, bias=False)  # 12
        # dense block 1 (3 layers)
        self.d1_1 = TinyDenseLayer(12, growth)
        self.d1_2 = TinyDenseLayer(12+growth, growth)
        self.d1_3 = TinyDenseLayer(12+2*growth, growth)
        self.trans1 = nn.Sequential(nn.Conv2d(12+3*growth, 20, 1, bias=False), nn.AvgPool2d(2))
        # dense block 2 (2 layers)
        self.d2_1 = TinyDenseLayer(20, growth)
        self.d2_2 = TinyDenseLayer(20+growth, growth)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(20+2*growth, 10)

    def forward(self, x):
        x = self.init(x)
        x = self.d1_1(x)
        x = self.d1_2(x)
        x = self.d1_3(x)
        x = self.trans1(x)
        x = self.d2_1(x)
        x = self.d2_2(x)
        x = self.pool(x).view(x.size(0), -1)
        return self.fc(x)

# --------------------------
# 7) TinyEffNet (mini MBConv with expansion)
# --------------------------
class MBConv(nn.Module):
    def __init__(self, in_ch, out_ch, exp=4, stride=1):
        super().__init__()
        mid = in_ch * exp
        self.expand = nn.Sequential(
            nn.Conv2d(in_ch, mid, 1, bias=False),
            nn.BatchNorm2d(mid),
            nn.ReLU()
        )
        self.dw = nn.Sequential(
            nn.Conv2d(mid, mid, 3, stride=stride, padding=1, groups=mid, bias=False),
            nn.BatchNorm2d(mid),
            nn.ReLU()
        )
        self.project = nn.Sequential(
            nn.Conv2d(mid, out_ch, 1, bias=False),
            nn.BatchNorm2d(out_ch)
        )
        self.use_res = (in_ch == out_ch and stride == 1)
    def forward(self, x):
        out = self.expand(x)
        out = self.dw(out)
        out = self.project(out)
        if self.use_res:
            return out + x
        return out

class TinyEffNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Conv2d(1, 8, 3, padding=1, bias=False)
        self.mb1 = MBConv(8, 16, exp=2, stride=1)
        self.pool = nn.MaxPool2d(2)   # 16x14x14
        self.mb2 = MBConv(16, 24, exp=2, stride=1)
        self.pool2 = nn.MaxPool2d(2)  # 24x7x7
        self.head = nn.Sequential(nn.Flatten(), nn.Linear(24*7*7, 64), nn.ReLU(), nn.Linear(64, 10))
    def forward(self, x):
        x = self.stem(x)
        x = self.mb1(x)
        x = self.pool(x)
        x = self.mb2(x)
        x = self.pool2(x)
        return self.head(x)

# --------------------------
# 8) FireNet-Mini (mini SqueezeNet Fire)
# --------------------------
class Fire(nn.Module):
    def __init__(self, in_ch, squeeze_ch, expand_ch):
        super().__init__()
        self.squeeze = nn.Conv2d(in_ch, squeeze_ch, 1)
        self.expand1 = nn.Conv2d(squeeze_ch, expand_ch, 1)
        self.expand3 = nn.Conv2d(squeeze_ch, expand_ch, 3, padding=1)
        self.relu = nn.ReLU()
    def forward(self, x):
        s = self.relu(self.squeeze(x))
        e = torch.cat([self.relu(self.expand1(s)), self.relu(self.expand3(s))], dim=1)
        return e

class FireNetMini(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Conv2d(1, 8, 3, padding=1)
        self.fire1 = Fire(8, 4, 4)   # -> 8
        self.pool = nn.MaxPool2d(2)  # 8x14x14
        self.fire2 = Fire(8, 6, 6)   # -> 12
        self.pool2 = nn.MaxPool2d(2) # -> 12x7x7
        self.fc = nn.Sequential(nn.Flatten(), nn.Linear(12*7*7, 64), nn.ReLU(), nn.Linear(64, 10))
    def forward(self, x):
        x = self.stem(x)
        x = self.fire1(x)
        x = self.pool(x)
        x = self.fire2(x)
        x = self.pool2(x)
        return self.fc(x)

# --------------------------
# Model factory
# --------------------------
MODEL_REGISTRY = {
    "nanocnn": NanoCNN,
    "picocnn": PicoCNN,
    "minimobilenet": MiniMobileNet,
    "mobilenano": MobileNetNano,
    "tinyshufflenet": TinyShuffleNet,
    "microdensenet": MicroDenseNet,
    "tinyeffnet": TinyEffNet,
    "firenetmini": FireNetMini
}

def make_model(name):
    assert name in MODEL_REGISTRY, f"Unknown model {name}"
    return MODEL_REGISTRY[name]()

# --------------------------
# Train & Eval helpers
# --------------------------
def train_one_epoch(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0.0
    correct = 0
    n = 0
    for X, y in loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(X)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X.size(0)
        preds = out.argmax(dim=1)
        correct += (preds == y).sum().item()
        n += X.size(0)
    return total_loss / n, correct / n

def evaluate(model, loader, loss_fn=None):
    model.eval()
    correct = 0
    total = 0
    losses = 0.0
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            out = model(X)
            if loss_fn:
                losses += loss_fn(out, y).item() * X.size(0)
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += X.size(0)
    return (losses / total) if loss_fn else None, correct / total

# --------------------------
# Run / orchestrate
# --------------------------
def run_model(name, epochs=EPOCHS, verbose=True):
    model = make_model(name).to(device)
    p = count_params(model)
    print(f"\n=== Model: {name} | params = {p:,}")
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LR)

    history = {"train_loss":[], "train_acc":[], "val_loss":[], "val_acc":[]}
    t0 = time.time()
    for ep in range(1, epochs+1):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, loss_fn)
        val_loss, val_acc = evaluate(model, test_loader, loss_fn)
        history["train_loss"].append(tr_loss); history["train_acc"].append(tr_acc)
        history["val_loss"].append(val_loss); history["val_acc"].append(val_acc)
        if verbose:
            print(f"Ep {ep}/{epochs} | train_loss={tr_loss:.4f} train_acc={tr_acc:.4f} | val_loss={val_loss:.4f} val_acc={val_acc:.4f}")
    dt = time.time() - t0
    print(f"Finished {name} in {dt:.1f}s | final val_acc={history['val_acc'][-1]:.4f}")
    return model, history

# --------------------------
# Interactive: choose models to run
# --------------------------
print("\nAvailable models:")
for k in MODEL_REGISTRY.keys():
    print(" -", k)

# Example usage:
# 1) Run a single model:
# model, hist = run_model("nanocnn", epochs=3)
#
# 2) Run all models sequentially (careful: will take longer):
# results = {}
# for key in MODEL_REGISTRY.keys():
#     m, h = run_model(key, epochs=2)
#     results[key] = (m, h)
#
# Uncomment below to run all with 2 epochs each (recommended if you want quick comparison):
#
#results = {}
#for key in MODEL_REGISTRY.keys():
#    results[key] = run_model(key, epochs=2)
#
# Or choose one model to run now:



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5

In [ ]:
selected = "nanocnn"   # change to any name above, e.g. "microdensenet"
model, history = run_model(selected, epochs=EPOCHS)


=== Model: nanocnn | params = 52,138


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Ep 1/5 | train_loss=0.3154 train_acc=0.9068 | val_loss=0.0986 val_acc=0.9686
Ep 2/5 | train_loss=0.0859 train_acc=0.9747 | val_loss=0.0618 val_acc=0.9805
Ep 3/5 | train_loss=0.0628 train_acc=0.9811 | val_loss=0.0572 val_acc=0.9814
Ep 4/5 | train_loss=0.0505 train_acc=0.9846 | val_loss=0.0421 val_acc=0.9863
Ep 5/5 | train_loss=0.0411 train_acc=0.9878 | val_loss=0.0375 val_acc=0.9868
Finished nanocnn in 149.3s | final val_acc=0.9868


In [ ]:
# Colab-ready: Train 14 lightweight CNN models on MNIST for 10 epochs each
!pip install -q torch torchvision --upgrade

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# --------------------------
# Hyperparameters
# --------------------------
BATCH_SIZE = 128
EPOCHS = 10
LR = 1e-3

# --------------------------
# MNIST Dataset
# --------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_ds = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_ds  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=1000, shuffle=False, num_workers=2, pin_memory=True)

# --------------------------
# Utility: count parameters
# --------------------------
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# --------------------------
# Model Definitions
# --------------------------

# 1) NanoCNN
class NanoCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(8, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten()
        )
        self.classifier = nn.Sequential(
            nn.Linear(16*7*7, 64), nn.ReLU(), nn.Linear(64, 10)
        )
    def forward(self, x): return self.classifier(self.features(x))

# 2) PicoCNN
class PicoCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1,6,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(6,12,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(12*7*7,40), nn.ReLU(),
            nn.Linear(40,10)
        )
    def forward(self,x): return self.net(x)

# 3) MiniMobileNet
def depthwise_conv(in_ch,out_ch,stride=1):
    return nn.Sequential(
        nn.Conv2d(in_ch,in_ch,3,stride=stride,padding=1,groups=in_ch,bias=False),
        nn.BatchNorm2d(in_ch), nn.ReLU(),
        nn.Conv2d(in_ch,out_ch,1,bias=False),
        nn.BatchNorm2d(out_ch), nn.ReLU()
    )
class MiniMobileNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Sequential(nn.Conv2d(1,8,3,padding=1,bias=False), nn.BatchNorm2d(8), nn.ReLU())
        self.ds1 = depthwise_conv(8,16)
        self.ds2 = depthwise_conv(16,24)
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Sequential(nn.Flatten(), nn.Linear(24*7*7,64), nn.ReLU(), nn.Linear(64,10))
    def forward(self,x):
        x = self.stem(x)
        x = self.ds1(x)
        x = self.pool(x)
        x = self.ds2(x)
        x = self.pool(x)
        return self.fc(x)

# 4) MobileNet-Nano (smaller version)
class MobileNetNano(MiniMobileNet): pass

# 5) TinyShuffleNet
def channel_shuffle(x,groups):
    b,c,h,w = x.size()
    cpg = c // groups
    x = x.view(b,groups,cpg,h,w)
    x = x.transpose(1,2).contiguous()
    return x.view(b,-1,h,w)
class ShuffleUnit(nn.Module):
    def __init__(self,in_c,out_c,groups=2):
        super().__init__()
        mid_c = out_c//2
        self.conv_reduce = nn.Conv2d(in_c,mid_c,1,bias=False)
        self.dw = nn.Conv2d(mid_c,mid_c,3,padding=1,groups=mid_c,bias=False)
        self.conv_expand = nn.Conv2d(mid_c,out_c,1,bias=False)
        self.bn = nn.BatchNorm2d(out_c)
        self.relu = nn.ReLU()
        self.groups = groups
    def forward(self,x):
        x = self.conv_reduce(x)
        x = self.dw(x)
        x = self.conv_expand(x)
        x = self.bn(x)
        x = self.relu(x)
        return channel_shuffle(x,self.groups)
class TinyShuffleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Conv2d(1,12,3,padding=1)
        self.stage1 = ShuffleUnit(12,24)
        self.stage2 = ShuffleUnit(24,32)
        self.pool = nn.MaxPool2d(2)
        self.pool2 = nn.MaxPool2d(2)
        self.fc = nn.Sequential(nn.Flatten(), nn.Linear(32*7*7,64), nn.ReLU(), nn.Linear(64,10))
    def forward(self,x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.pool(x)
        x = self.stage2(x)
        x = self.pool2(x)
        return self.fc(x)

# 6) MicroDenseNet
class TinyDenseLayer(nn.Module):
    def __init__(self, in_ch, growth):
        super().__init__()
        self.bn = nn.BatchNorm2d(in_ch)
        self.conv = nn.Conv2d(in_ch, growth, 3, padding=1, bias=False)
        self.relu = nn.ReLU()
    def forward(self, x):
        out = self.conv(self.relu(self.bn(x)))
        return torch.cat([x, out], 1)
class MicroDenseNet(nn.Module):
    def __init__(self, growth=8):
        super().__init__()
        self.init = nn.Conv2d(1,12,3,padding=1,bias=False)
        self.d1_1 = TinyDenseLayer(12,growth)
        self.d1_2 = TinyDenseLayer(12+growth,growth)
        self.d1_3 = TinyDenseLayer(12+2*growth,growth)
        self.trans1 = nn.Sequential(nn.Conv2d(12+3*growth,20,1,bias=False), nn.AvgPool2d(2))
        self.d2_1 = TinyDenseLayer(20,growth)
        self.d2_2 = TinyDenseLayer(20+growth,growth)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(20+2*growth,10)
    def forward(self,x):
        x = self.init(x)
        x = self.d1_1(x)
        x = self.d1_2(x)
        x = self.d1_3(x)
        x = self.trans1(x)
        x = self.d2_1(x)
        x = self.d2_2(x)
        x = self.pool(x).view(x.size(0),-1)
        return self.fc(x)

# 7) TinyEffNet
class MBConv(nn.Module):
    def __init__(self, in_ch, out_ch, exp=4, stride=1):
        super().__init__()
        mid = in_ch * exp
        self.expand = nn.Sequential(nn.Conv2d(in_ch,mid,1,bias=False), nn.BatchNorm2d(mid), nn.ReLU())
        self.dw = nn.Sequential(nn.Conv2d(mid,mid,3,stride=stride,padding=1,groups=mid,bias=False),
                                nn.BatchNorm2d(mid), nn.ReLU())
        self.project = nn.Sequential(nn.Conv2d(mid,out_ch,1,bias=False), nn.BatchNorm2d(out_ch))
        self.use_res = (in_ch==out_ch and stride==1)
    def forward(self,x):
        out = self.expand(x)
        out = self.dw(out)
        out = self.project(out)
        if self.use_res: return out + x
        return out
class TinyEffNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Conv2d(1,8,3,padding=1,bias=False)
        self.mb1 = MBConv(8,16,exp=2)
        self.pool = nn.MaxPool2d(2)
        self.mb2 = MBConv(16,24,exp=2)
        self.pool2 = nn.MaxPool2d(2)
        self.head = nn.Sequential(nn.Flatten(), nn.Linear(24*7*7,64), nn.ReLU(), nn.Linear(64,10))
    def forward(self,x):
        x = self.stem(x)
        x = self.mb1(x)
        x = self.pool(x)
        x = self.mb2(x)
        x = self.pool2(x)
        return self.head(x)

# 8) FireNetMini
class Fire(nn.Module):
    def __init__(self,in_ch,sq_ch,exp_ch):
        super().__init__()
        self.squeeze = nn.Conv2d(in_ch,sq_ch,1)
        self.expand1 = nn.Conv2d(sq_ch,exp_ch,1)
        self.expand3 = nn.Conv2d(sq_ch,exp_ch,3,padding=1)
        self.relu = nn.ReLU()
    def forward(self,x):
        s = self.relu(self.squeeze(x))
        e = torch.cat([self.relu(self.expand1(s)),self.relu(self.expand3(s))],1)
        return e
class FireNetMini(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Conv2d(1,8,3,padding=1)
        self.fire1 = Fire(8,4,4)
        self.pool = nn.MaxPool2d(2)
        self.fire2 = Fire(8,6,6)
        self.pool2 = nn.MaxPool2d(2)
        self.fc = nn.Sequential(nn.Flatten(), nn.Linear(12*7*7,64), nn.ReLU(), nn.Linear(64,10))
    def forward(self,x):
        x = self.stem(x)
        x = self.fire1(x)
        x = self.pool(x)
        x = self.fire2(x)
        x = self.pool2(x)
        return self.fc(x)

# 9) TinyCNN
class TinyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1,6,5), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(6,16,5), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(16*4*4,120), nn.ReLU(),
            nn.Linear(120,84), nn.ReLU(),
            nn.Linear(84,10)
        )
    def forward(self,x): return self.net(x)

# 10) MicroCNN (simpler than TinyCNN)
class MicroCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1,4,3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(4,8,3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(8*5*5,32), nn.ReLU(),
            nn.Linear(32,10)
        )
    def forward(self,x): return self.net(x)

# 11) DSConv
class DSConv(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1,4,3,padding=1), nn.ReLU(),
            nn.Conv2d(4,4,3,padding=1,groups=4), nn.ReLU(),
            nn.Conv2d(4,8,1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(8*14*14,10)
        )
    def forward(self,x): return self.net(x)

# 12) SqueezeTiny
class SqueezeTiny(nn.Module):
    def __init__(self):
        super().__init__()
        self.squeeze = nn.Conv2d(1,4,1)
        self.expand = nn.Conv2d(4,8,3,padding=1)
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Sequential(nn.Flatten(), nn.Linear(8*14*14,10))
    def forward(self,x):
        x = self.squeeze(x)
        x = self.expand(x)
        x = self.pool(x)
        return self.fc(x)

# 13) TinyResNet
class BasicBlock(nn.Module):
    def __init__(self,in_ch,out_ch):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch,out_ch,3,padding=1)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_ch,out_ch,3,padding=1)
        self.bn2 = nn.BatchNorm2d(out_ch)
    def forward(self,x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += x
        return self.relu(out)
class TinyResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Conv2d(1,8,3,padding=1)
        self.block1 = BasicBlock(8,8)
        self.pool = nn.MaxPool2d(2)
        self.block2 = BasicBlock(8,8)
        self.pool2 = nn.MaxPool2d(2)
        self.fc = nn.Sequential(nn.Flatten(), nn.Linear(8*7*7,10))
    def forward(self,x):
        x = self.stem(x)
        x = self.block1(x)
        x = self.pool(x)
        x = self.block2(x)
        x = self.pool2(x)
        return self.fc(x)

# 14) TinyGoogLeNet
class InceptionTiny(nn.Module):
    def __init__(self,in_ch,out1,out3,out5):
        super().__init__()
        self.b1 = nn.Sequential(nn.Conv2d(in_ch,out1,1), nn.ReLU())
        self.b3 = nn.Sequential(nn.Conv2d(in_ch,out3,3,padding=1), nn.ReLU())
        self.b5 = nn.Sequential(nn.Conv2d(in_ch,out5,5,padding=2), nn.ReLU())
    def forward(self,x):
        return torch.cat([self.b1(x),self.b3(x),self.b5(x)],1)
class TinyGoogLeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = nn.Conv2d(1,16,3,padding=1)
        self.inc1 = InceptionTiny(16,8,8,8)
        self.pool = nn.MaxPool2d(2)
        self.inc2 = InceptionTiny(24,12,12,12)
        self.fc = nn.Linear(36*14*14,10)
    def forward(self,x):
        x = self.stem(x)
        x = self.inc1(x)
        x = self.pool(x)
        x = self.inc2(x)
        x = x.view(x.size(0),-1)
        return self.fc(x)

# 15) LeNet
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1,6,5), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(6,16,5), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(16*4*4,120), nn.ReLU(),
            nn.Linear(120,84), nn.ReLU(),
            nn.Linear(84,10)
        )
    def forward(self,x): return self.net(x)

# --------------------------
# Model registry
# --------------------------
MODEL_REGISTRY = {
    "nanocnn": NanoCNN,
    "picocnn": PicoCNN,
    "minimobilenet": MiniMobileNet,
    "mobilenano": MobileNetNano,
    "tinyshufflenet": TinyShuffleNet,
    "microdensenet": MicroDenseNet,
    "tinyeffnet": TinyEffNet,
    "firenetmini": FireNetMini,
    "tinycnn": TinyCNN,
    "microcnn": MicroCNN,
    "dsconv": DSConv,
    "squeezetiny": SqueezeTiny,
    "tinyresnet": TinyResNet,
    "tinygooglenet": TinyGoogLeNet,
    "lenet": LeNet
}

# --------------------------
# Training helpers
# --------------------------
def train_one_epoch(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    correct = 0
    n = 0
    for X, y in loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(X)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()*X.size(0)
        correct += (out.argmax(1)==y).sum().item()
        n += X.size(0)
    return total_loss/n, correct/n

def evaluate(model, loader, loss_fn=None):
    model.eval()
    correct = 0
    total = 0
    losses = 0
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            out = model(X)
            if loss_fn: losses += loss_fn(out,y).item()*X.size(0)
            correct += (out.argmax(1)==y).sum().item()
            total += X.size(0)
    return (losses/total if loss_fn else None), correct/total

# --------------------------
# Run all models sequentially
# --------------------------
for name, cls in MODEL_REGISTRY.items():
    print(f"\n=== Training {name} ===")
    model = cls().to(device)
    print(f"Params: {count_params(model):,}")
    optimizer = optim.Adam(model.parameters(), lr=LR)
    loss_fn = nn.CrossEntropyLoss()
    start_time = time.time()
    for ep in range(1,EPOCHS+1):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, loss_fn)
        val_loss, val_acc = evaluate(model, test_loader, loss_fn)
        print(f"Epoch {ep}/{EPOCHS}: train_acc={tr_acc:.4f}, val_acc={val_acc:.4f}")
    print(f"Final validation accuracy for {name}: {val_acc:.4f}")
    print("Time elapsed: %.2f sec\n" % (time.time()-start_time))


Device: cpu

=== Training nanocnn ===
Params: 52,138
Epoch 1/10: train_acc=0.9121, val_acc=0.9690
Epoch 2/10: train_acc=0.9717, val_acc=0.9773
Epoch 3/10: train_acc=0.9804, val_acc=0.9819
Epoch 4/10: train_acc=0.9846, val_acc=0.9848
Epoch 5/10: train_acc=0.9860, val_acc=0.9845
Epoch 6/10: train_acc=0.9888, val_acc=0.9827
Epoch 7/10: train_acc=0.9898, val_acc=0.9864
Epoch 8/10: train_acc=0.9906, val_acc=0.9862
Epoch 9/10: train_acc=0.9916, val_acc=0.9867
Epoch 10/10: train_acc=0.9934, val_acc=0.9871
Final validation accuracy for nanocnn: 0.9871
Time elapsed: 281.68 sec


=== Training picocnn ===
Params: 24,690
Epoch 1/10: train_acc=0.8889, val_acc=0.9620
Epoch 2/10: train_acc=0.9681, val_acc=0.9768
Epoch 3/10: train_acc=0.9769, val_acc=0.9815
Epoch 4/10: train_acc=0.9804, val_acc=0.9813
Epoch 5/10: train_acc=0.9828, val_acc=0.9845
Epoch 6/10: train_acc=0.9848, val_acc=0.9836
Epoch 7/10: train_acc=0.9867, val_acc=0.9858
Epoch 8/10: train_acc=0.9880, val_acc=0.9842
Epoch 9/10: train_acc=0